In [38]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

In [39]:
sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

mktdir = "./MkTdata"

# force=True read from alphavantage server
# force=False try to read first from the local directory if data exists if not from alphavantage
mktdata = az.readMkT(symb, dstart = sdate, dend = edate, 
                     dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


In [40]:
alpha_SMCR95 = [0.95]
alpha_SMCR90 = [0.9]
alpha_SMCR85 = [0.85]
alpha_SMCR = [0.95, 0.90, 0.85]

coef_LSSD1 = [1.]
coef_LSSD2 = [0.5] * 2
coef_LSSD3 = [1./3] * 3

rtype = 'InvNrisk'

hlength1 = 1.25
hlength2 = 2.25

models = {'P_SMCR95': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR95, 'rtype': rtype, 'hlength': hlength1}},
          'P_SMCR90': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR90, 'rtype': rtype, 'hlength': hlength1}},
          'P_SMCR85': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR85, 'rtype': rtype, 'hlength': hlength1}},
          'P_SMCR': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength1}},
          'P_LSSD1': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD1, 'rtype': rtype, 'hlength': hlength1}},
          'P_LSSD2': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD2, 'rtype': rtype, 'hlength': hlength1}},
          'P_LSSD3': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD3, 'rtype': rtype, 'hlength': hlength1}},
          'P_SMCR95L': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR95, 'rtype': rtype, 'hlength': hlength2}},
          'P_SMCR90L': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR90, 'rtype': rtype, 'hlength': hlength2}},
          'P_SMCR85L': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR85, 'rtype': rtype, 'hlength': hlength2}},
          'P_SMCRL': {'type': 'Port_SMCR', 'm_param': {'mu': 0., 'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength2}},
          'P_LSSD1L': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD1, 'rtype': rtype, 'hlength': hlength2}},
          'P_LSSD2L': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD2, 'rtype': rtype, 'hlength': hlength2}},
          'P_LSSD3L': {'type': 'Port_LSSD', 'm_param': {'mu': 0., 'coef': coef_LSSD3, 'rtype': rtype, 'hlength': hlength2}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

In [41]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

In [42]:
pdata = pd.concat(port, axis=1, join='inner').melt(var_name='symbol', value_name='close', ignore_index=False)

In [43]:
ps = az.Port_Simple(pdata, col='close', pname='ALL')
ports = ps.set_model()

In [44]:
_ = ps.port_view_all(sdate=pd.to_datetime('2000-01-01'), componly=True, fancy=True)

In [45]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,DD_date,DD_start,DD_end
symbol,,,,,
P_LSSD1,12.63,-24.89,2015-09-28,2015-03-20,2017-10-27
P_LSSD1L,12.70,-23.25,2018-12-24,2018-09-14,2019-11-25
P_LSSD2,12.68,-24.43,2016-02-09,2015-03-20,2017-10-27
P_LSSD2L,12.78,-23.13,2018-12-24,2018-09-14,2019-08-08
P_LSSD3,12.79,-24.65,2016-02-09,2015-03-20,2017-10-27
P_LSSD3L,12.72,-23.08,2018-12-24,2018-09-14,2019-08-08
P_N,14.87,-17.43,2020-03-18,2020-02-19,2020-04-14
P_SMCR,12.97,-25.30,2016-02-09,2015-03-20,2017-10-27
P_SMCR85,12.72,-26.34,2016-02-09,2015-03-20,2017-10-31


In [46]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_LSSD1,P_LSSD1L,P_LSSD2,P_LSSD2L,P_LSSD3,P_LSSD3L,P_N,P_SMCR,P_SMCR85,P_SMCR85L,P_SMCR90,P_SMCR90L,P_SMCR95,P_SMCR95L,P_SMCRL
year,,,,,,,,,,,,,,,
2014,12.17%,11.32%,12.09%,11.96%,11.96%,12.01%,6.94%,11.68%,12.38%,11.91%,12.19%,12.07%,11.55%,12.13%,12.16%
2015,-14.39%,-1.79%,-13.90%,-2.12%,-13.30%,-1.81%,0.60%,-12.62%,-14.32%,-1.51%,-12.78%,-1.15%,-11.22%,0.17%,-1.13%
2016,-1.99%,4.97%,-2.18%,6.28%,-2.34%,6.06%,8.32%,-2.32%,-2.81%,6.90%,-2.29%,7.39%,-3.95%,7.03%,7.03%
2017,32.74%,25.10%,32.51%,24.67%,32.08%,24.59%,21.57%,31.38%,31.79%,24.84%,31.48%,24.60%,31.41%,23.93%,24.54%
2018,10.16%,7.66%,9.52%,7.53%,9.66%,7.13%,7.69%,10.43%,9.74%,9.02%,10.34%,7.58%,10.13%,6.42%,6.92%
2019,19.42%,21.65%,19.73%,20.73%,20.17%,21.06%,25.06%,20.28%,20.71%,20.19%,20.46%,20.60%,20.69%,21.11%,21.12%
2020,39.32%,22.34%,39.73%,22.96%,39.61%,22.60%,32.55%,40.30%,41.43%,22.72%,40.60%,20.58%,39.00%,24.13%,21.73%
2021,-2.03%,-1.30%,-1.58%,-1.19%,-1.28%,-1.30%,1.63%,-1.17%,-1.79%,-1.19%,-1.23%,-0.77%,-0.37%,-1.18%,-1.06%
